In [1]:
# !python -m pip install statsmodels
# !python -m pip install plotly
# !pip install "nbformat>=4.2.0"

In [2]:
# 라이브러리 불러오기
import requests
import os
from time import sleep
import numpy as np
import pandas as pd
import numpy as np
import scipy.stats as stats
from math import ceil
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from statsmodels.stats.power import FTestAnovaPower, GofChisquarePower
from sklearn.model_selection import ParameterGrid
from scipy.stats import chi2_contingency
import plotly.express as px
import plotly.graph_objects as go

### 생각해둬야 할것.
* 샘플사이즈를 구한뒤에 아래 과정을 진행하는 것이 맞습니다.
  
* 이 데이터셋은 이미 테스트 사후에 확보된 것으로, 전체 데이터가 샘플데이터이고, 사후분석하는것이 맞으나,
* 테스트 사전에 샘플사이즈를 구하고 실험해서 얻은 테스트셋으로서,
* 샘플사이즈만큼 확보된 시점에 테스트를 중단하고 나머지 데이터셋은 저장하지 않았다고 간주하겠습니다.

## (테스트 설계) 샘플사이즈 계산

ABT

In [3]:
# 1.5% point의 전환율 상승의 비즈니스 목표설정이 있었으므로,
# 이에 맞춰 효과크기를 계신.
effect_size = sms.proportion_effectsize(0.05, 0.065)

sample_size = sms.NormalIndPower().solve_power(
    effect_size=effect_size, # 효과크기 
    power=0.8, # 검정력 80%
    alpha=0.05, # 유의수준 5%
    ratio=1 # 대조와 처치의 비율 1:1
)                                                  

sample_size = ceil(sample_size) # 샘플사이즈는 정수형이 맞으므로 소수점 제거

print(f"각 그룹별 필요 샘플사이즈 : {sample_size}")

각 그룹별 필요 샘플사이즈 : 3766


In [4]:
# 모표준편차가 3000원인 환경에서 모평균 12500원에서 200원 상승을 기대한다고 가정
# 이에 맞춰 효과크기를 계신.
effect_size = (12500 - 12700) / 3000
print("effect_size : ", effect_size)
sample_size = sms.TTestIndPower().solve_power(
    effect_size, # 효과크기 
    power=0.8, # 검정력 80%
    alpha=0.05, # 유의수준 5%
    ratio=1 # 대조와 처치의 비율 1:1
)                                                  

sample_size = ceil(sample_size) # 샘플사이즈는 정수형이 맞으므로 소수점 제거

print(f"각 그룹별 필요 샘플사이즈 : {sample_size}")



effect_size :  -0.06666666666666667
각 그룹별 필요 샘플사이즈 : 3533


* 모수를 가정하는 테스트는 위 방법으로 간단히 샘플사이즈 산출이 가능하지만,
  
* 정규성 가정, 등분산성 가정이 어려운 경우 비모수 검정을 위한 테스트 설계를 해야함.
* 이때는 python으로 관련 코드를 구현하는 것은 매우 어려운작업이므로, (보통 매우 큰 샘플사이즈가 필요하다. 모수가정보다 수천배)
* G-power 프로그램을 사용해서 샘플사이즈를 구하는 방법을 사용.
* https://www.psychologie.hhu.de/arbeitsgruppen/allgemeine-psychologie-und-arbeitspsychologie/gpower.html

MVT

* FWER (Family-Wise Error Rate) : 

  * $ \textcolor{yellow}{\text{FWER} = 1 - (1 - \alpha)^k} $ 
  
  * 여러 통계적 검정을 수행할 때, 하나 이상의 잘못된 양성(Type I 오류) 결과가 나올 확률
  * 다중 비교 상황에서 하나 이상의 검정이 유의하게 나올 확률을 제어하기 위해 사용

* 본페로니 교정 (Bonferroni Correction):
  
  * $ \textcolor{yellow}{\alpha_{adj} = \frac{\alpha}{m}}$
 
  *  본페로니 교정은  다중 비교  문제에서  유의수준  을  조정  하여  잘못된   양성(Type I  오류)  의  확률  을  줄이는 가장 보편하고 단순한 방법
  *  여러 통계적 검정을 수행할 때, 각 검정의 유의수준을 전체 유의수준을 유지하기 위해 나누어 적용
  *  예를 들어, 전체 유의수준이 0.05이고 5개의 검정을 수행하는 경우, 각 검정의 유의수준은 0.05 / 5 = 0.01
  *  이를 통해 다중 비교로 인한 잘못된 양성 오류를 줄임.

* 그외의 교정 방법들..  
  
  * Holm 교정 (Holm Correction):
    
    * Holm 교정은 본페로니 교정의 보수성을 완화한 방법으로, 단계별로 유의수준을 조정하여 다중 비교 문제를 해결
    * 각 검정의 p-value를 오름차순으로 정렬한 후, 가장 작은 p-value부터 순차적으로 유의수준을 조정
    * 예를 들어, 전체 유의수준이 0.05이고 5개의 검정을 수행하는 경우, 가장 작은 p-value는 0.05/5, 두 번째 작은 p-value는 0.05/4, ... , 마지막 p-value는 0.05로 비교
    * 이를 통해 다중 비교로 인한 잘못된 양성 오류를 줄임.

  * Hochberg 교정 (Hochberg Correction):
    
    * Hochberg 교정은 Holm 교정과 유사하지만, 단계별로 유의수준을 조정하는 방식이 반대
    * 각 검정의 p-value를 오름차순으로 정렬한 후, 가장 큰 p-value부터 순차적으로 유의수준을 조정
    * 예를 들어, 전체 유의수준이 0.05이고 5개의 검정을 수행하는 경우, 가장 큰 p-value는 0.05, 두 번째 큰 p-value는 0.05/2, ... , 가장 작은 p-value는 0.05/5로 비교
    * 이를 통해 다중 비교로 인한 잘못된 양성 오류를 줄임.

  * Benjamini-Hochberg 교정 (Benjamini-Hochberg Correction):
    
    * Benjamini-Hochberg 교정은 FDR (False Discovery Rate)을 제어하는 방법으로, 다중 비교 문제에서 잘못된 양성 오류의 비율을 줄이는 데 사용
    * 각 검정의 p-value를 오름차순으로 정렬한 후, 각 p-value에 대해 (i/m) * Q 값을 계산 (여기서 i는 p-value의 순위, m은 전체 검정 수, Q는 원하는 FDR)
    * 각 p-value와 해당 (i/m) * Q 값을 비교하여 유의한 p-value를 결정
    * 이를 통해 다중 비교로 인한 잘못된 양성 오류의 비율을 줄임.


In [5]:
# 이산형 (Chi-Square)
def estimate_sample_size_for_discrete_mvt(effect_size, alpha=0.05, power=0.8, num_tests=3):
    # 본페로니 교정 적용
    adjusted_alpha = alpha / num_tests
    
    # 검정력 분석 객체 생성
    power_analysis = GofChisquarePower()
    
    return ceil(power_analysis.solve_power(
        effect_size=effect_size,
        alpha=adjusted_alpha,
        power=power
    ))

In [6]:
sample_size = estimate_sample_size_for_discrete_mvt(effect_size=0.015, alpha=0.05, power=0.8, num_tests=3)
sample_size

46530

In [7]:
# 연속형 (ANOVA)
def estimate_sample_size_for_continuous_mvt(effect_size, alpha=0.05, power=0.8, num_tests=3):
    # 본페로니 교정 적용
    adjusted_alpha = alpha / (num_tests * (num_tests - 1) / 2)
    
    return ceil(FTestAnovaPower().solve_power(effect_size=effect_size, 
                                               alpha=adjusted_alpha, 
                                               k_groups=num_tests, 
                                               power=power))

In [10]:
sample_size = estimate_sample_size_for_continuous_mvt(effect_size=0.02, alpha=0.05, power=0.8, num_tests=3)
sample_size

31399

## (테스트시작) 데이터 수집

데이터 로드

In [9]:
# 데이터 로드
df = pd.read_csv('datasets/ab_test_data.csv')
print(df)

           uid                          ts grp algorithm  converted  amount
0       851227  2025-01-21 22:11:48.556739   A        v1          0       0
1       804351  2025-01-12 08:01:45.159739   A        v1          0       0
2       661713  2025-01-11 16:55:06.154213   B        v2          0   11762
3       853664  2025-01-08 18:28:03.143765   B        v2          0   14987
4       865098  2025-01-21 01:52:26.210827   A        v1          1   13058
...        ...                         ...  ..       ...        ...     ...
441440  445898  2025-01-14 16:32:53.398579   C        v3          0   11796
441441  208766  2025-01-16 20:56:38.511760   C        v3          0   12451
441442  381188  2025-01-14 23:56:34.436404   C        v3          0   14853
441443  601482  2025-01-11 14:06:36.667010   C        v3          0   12866
441444  522970  2025-01-14 00:18:56.099964   C        v3          0   11611

[441445 rows x 6 columns]


데이터 점검 및 오류데이터 정리

In [14]:
# A그룹에 algorithm이 v2, B그룹에 algorithm이 v1 이 할당되어 있다. (데이터수집 오류로 생각됨)
df.groupby(['grp', 'algorithm']).size().reset_index()

,grp,algorithm,0
0,A,v1,145274
1,A,v2,1928
2,B,v1,1965
3,B,v2,145311
4,C,v3,146967


In [15]:
# 데이터 오류로 판단할수 있으므로 제거
prep_df1 = pd.concat([
    df.query("grp == 'A' and algorithm == 'v1'"), 
    df.query("grp == 'B' and algorithm == 'v2'"),
    df.query("grp == 'C' and algorithm == 'v3'")
])
print(f"df.shape : {df.shape} => prep_df.shape : {prep_df1.shape} 제거한 행 수: {df.shape[0] - prep_df1.shape[0]}")


df.shape : (441445, 6) => prep_df.shape : (437552, 6) 제거한 행 수: 3893


In [16]:
# 중복 사용자 제거 (이번 테스트는 샘플사이즈 자체가 확보한 사용자 이기 때문)
prep_df2 = prep_df1.drop_duplicates(subset= ['uid'], keep='first')
print(f"prep_df1.shape : {prep_df1.shape} => prep_df2.shape : {prep_df2.shape} 제거된 행수: {prep_df1.shape[0] - prep_df2.shape[0]}")

prep_df1.shape : (437552, 6) => prep_df2.shape : (437486, 6) 제거된 행수: 66


샘플사이즈가 확보된 날짜까지의 데이터만 별도 저장

In [17]:
def get_filtered_df(df, sample_size):
    df.ts = pd.to_datetime(df.ts)
    sorted_df = df.sort_values('ts')
    start_time = df.ts.min()
    day = 1
    while True:
        end_time = start_time + pd.Timedelta(days=day)
        filtered_df = sorted_df[(sorted_df['ts'] >= start_time) & (sorted_df['ts'] <= end_time)]
        A_sample_size = filtered_df[filtered_df['grp'] == 'A'].shape[0]
        B_sample_size = filtered_df[filtered_df['grp'] == 'B'].shape[0]
        C_sample_size = filtered_df[filtered_df['grp'] == 'C'].shape[0]
        print(f"{day}일차 : 확보된 그룹별 샘플사이즈 (A:{A_sample_size}, B:{B_sample_size}, C:{C_sample_size})")
        if A_sample_size >= sample_size and B_sample_size >= sample_size and C_sample_size >= sample_size:
            print('테스트 중단!')
            break
        day += 1
        sleep(1)
    return filtered_df

In [18]:
# 필요샘플사이즈는 각 4433명, 총 8866명 이지만, 
# 안정적인 테스트 결과를 위해 각 30000명의 사용자를 확보해야 한다고 가정한다.
sample_df = get_filtered_df(prep_df2, 30000)


/tmp/ipykernel_174/3501489090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ts = pd.to_datetime(df.ts)


1일차 : 확보된 그룹별 샘플사이즈 (A:6655, B:6579, C:6744)
2일차 : 확보된 그룹별 샘플사이즈 (A:13141, B:13145, C:13307)
3일차 : 확보된 그룹별 샘플사이즈 (A:19722, B:19701, C:19969)
4일차 : 확보된 그룹별 샘플사이즈 (A:26159, B:26383, C:26489)
5일차 : 확보된 그룹별 샘플사이즈 (A:32844, B:32977, C:33241)
테스트 중단!


In [19]:
print(sample_df)

           uid                         ts grp algorithm  converted  amount
325747  425323 2025-01-02 13:32:15.234051   C        v3          0   12780
296988  110858 2025-01-02 13:33:03.767329   C        v3          0   12333
420164  179783 2025-01-02 13:36:06.600508   C        v3          0   10712
332968  437507 2025-01-02 13:37:28.246588   C        v3          1   11101
384759  192894 2025-01-02 13:37:52.169597   C        v3          1   11191
...        ...                        ...  ..       ...        ...     ...
76325   783818 2025-01-07 13:31:52.131433   B        v2          0   12899
142141  868674 2025-01-07 13:31:56.032796   B        v2          0   14142
194349  671188 2025-01-07 13:31:58.378189   B        v2          0   11378
137267  706869 2025-01-07 13:32:04.099623   B        v2          0   12397
118388  871556 2025-01-07 13:32:06.829080   B        v2          0   13445

[99062 rows x 6 columns]


## (테스트 분석) 이산형

In [20]:
# 분할표 만들기
contingency_table = sample_df.pivot_table(index='grp', columns='converted', aggfunc='size')
contingency_table

converted,0,1
grp,,
A,28917,3927
B,29102,3875
C,29299,3942


ABT - pair-wise

In [21]:
# group2 - group 1 신뢰구간 계산하기
def calc_proportion_test_ci(n1, p1, n2, p2, alpha=0.05):
    # minitab : https://support.minitab.com/ko-kr/minitab/18/help-and-how-to/statistics/basic-statistics/how-to/2-proportions/methods-and-formulas/methods-and-formulas/
    se = np.sqrt( p1 * (1 - p1) / n1 + p2 * (1 - p2) / n2 )
    diff_prop = p2 - p1
    cl_ll, cl_ul = diff_prop + np.array([-1, 1]) * stats.norm.ppf(1 - alpha / 2) * se
    return cl_ll, cl_ul

In [22]:
def analysis_pair_wise_abt(df, group1, group2):
    filtered_df = df[(df['grp'] == group1) | (df['grp'] == group2)]
    group1_size = filtered_df[filtered_df['grp'] == group1]['converted'].values.size
    group1_cvr = filtered_df[filtered_df['grp'] == group1]['converted'].values.sum() / group1_size
    group2_size = filtered_df[filtered_df['grp'] == group2]['converted'].values.size
    group2_cvr = filtered_df[filtered_df['grp'] == group2]['converted'].values.sum() / group2_size

    diff_mean = group2_cvr - group1_cvr
    # 신뢰구간 계산
    cl_ll, cl_ul = calc_proportion_test_ci(group1_size, group1_cvr, group2_size, group2_cvr)
    # 두비율 검정
    z_stat, pval = proportions_ztest([group1_cvr * group1_size, group2_cvr * group2_size], nobs=[group1_size, group2_size])
    
    return {
        'group1': group1,
        'group2': group2,
        '평균차이': diff_mean,
        '신뢰구간_하한': cl_ll,
        '신뢰구간_상한': cl_ul,
        'z_통계량': z_stat,
        'p값': pval
    }



In [23]:
# 그룹 조합 리스트 생성
from itertools import combinations
groups = sorted(sample_df['grp'].unique().tolist())
pairwise_combinations = list(combinations(groups, 2))
pairwise_combinations


[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [24]:
stats_df = pd.DataFrame([
    analysis_pair_wise_abt(sample_df, *comb) 
    for comb in pairwise_combinations
])
stats_df

,group1,group2,평균차이,신뢰구간_하한,신뢰구간_상한,z_통계량,p값
0,A,B,-0.002059,-0.006998,0.002880,0.817147,0.413844
1,A,C,-0.000977,-0.005916,0.003962,0.387622,0.698295
2,B,C,0.001082,-0.003833,0.005998,-0.431587,0.666042


MVT - 사후분석

In [25]:
# 카이제곱 검정 수행
from statsmodels.sandbox.stats.multicomp import multipletests
contingency_table = pd.crosstab(sample_df['grp'], sample_df['converted'])
contingency_table

converted,0,1
grp,,
A,28917,3927
B,29102,3875
C,29299,3942


In [26]:
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
rejects, _, _, _ = multipletests(stats_df['p값'], method='bonferroni', alpha=0.05)

# 결과 출력
print(f"Chi2 통계량: {chi2}")
print(f"p값: {p}")
if rejects.all():
    print("귀무가설 기각: 전체 그룹 간의 차이가 통계적으로 유의미")
else:
    print("귀무가설 채택: 전체 그룹 간의 차이가 통계적으로 유의미하지 않음")

Chi2 통계량: 0.6682765286143721
p값: 0.7159547843911922
귀무가설 채택: 전체 그룹 간의 차이가 통계적으로 유의미하지 않음


## (테스트 분석) 연속형

<img src="images/ref1.png" width="500" height="400">

* 온라인 AB테스트의 경우, 정규성 및 등분산성 가정이 필요 없을 만큼 데이터가 충분한 경우가 대부분이라 생략 가능하지만,
  
* 확보된 데이터가 5000이하로 적을 경우 통계 검정 정석대로 진행한다.

정규성 검정

* N 이 5000 이하이면 샤피로 검정으로 전환여부 데이터 분포에 대한 정규분포 가정이 가능한지 검정해야함
  
* 이번예제는 N > 5000 이므로 생략 가능
* 아래는 예시 코드

In [27]:
# Shapiro-Wilk 정규성 검정 수행
stat, p_value = stats.shapiro(sample_df.amount)

# 결과 출력
print(f"Shapiro-Wilk Test Statistic: {stat}")
print(f"P-value: {p_value}")

# 유의 수준 설정
alpha = 0.05

# 가설 검정
if p_value > alpha:
    print("귀무가설 채택: 데이터가 정규분포를 따름")
else:
    print("귀무가설 기각: 데이터가 정규분포를 따르지 않음")

Shapiro-Wilk Test Statistic: 0.7060848474502563
P-value: 0.0
귀무가설 기각: 데이터가 정규분포를 따르지 않음


/opt/conda/lib/python3.11/site-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


등분산성 검정
* 정규성을 만족하지 않으면 등분산성을 따지지 않는다.
  
* 2가지 테스트가 있다.
* Levene Test, Brown-Forsythe Test
* 두 테스트 중 최소 한개 이상 만족하면 등분산성을 인정한다.
* 아래는 예시 코드

In [28]:
data = sample_df.groupby('grp')['amount'].agg(lambda x : list(x)).tolist()

In [29]:
stat, p1 = stats.levene(*data, center='mean')
stat, p2 = stats.levene(*data, center='median')
print(p1, p2)
if p1 < alpha or p2 < alpha:
    print("등분산성 채택: 데이터가 등분산성 존재")
else:
    print("등분산성 기각: 데이터가 등분산성 없음 ")

0.0 9.639221008864368e-274
등분산성 채택: 데이터가 등분산성 존재


본 검정

* 모집단 정규 분포 가정 (모수적 방법)

In [32]:
def analysis_pair_wise_abt_by_parametric(df, group1, group2, alpha=0.05, eqvar=True):
    filtered_df = df[(df['grp'] == group1) | (df['grp'] == group2)]
    n1 = filtered_df[filtered_df['grp'] == group1]['converted'].values.size
    n2 = filtered_df[filtered_df['grp'] == group2]['converted'].values.size
    mu1 = filtered_df[filtered_df['grp'] == group1]['converted'].values.sum() / n1
    mu2 = filtered_df[filtered_df['grp'] == group2]['converted'].values.sum() / n2
    std1 = np.sqrt(filtered_df[filtered_df['grp'] == group1]['converted'].values.sum() / n1)
    std2 = np.sqrt(filtered_df[filtered_df['grp'] == group2]['converted'].values.sum() / n2)
    
    if eqvar:
        # 등분산성을 만족하는 경우, Student's t-test 수행
        t_stat, pval = stats.ttest_ind(
            filtered_df[filtered_df['grp'] == group1]['converted'],
            filtered_df[filtered_df['grp'] == group2]['converted'],
            equal_var=True
        )
    else:
        # 등분산성을 만족하지 않는 경우, Welch's t-test 수행
        t_stat, pval = stats.ttest_ind(
            filtered_df[filtered_df['grp'] == group1]['converted'],
            filtered_df[filtered_df['grp'] == group2]['converted'],
            equal_var=False
        )
        
    diff_mean = mu1 - mu2
    se_diff= np.sqrt((std1**2 / n1) + (std2**2 / n2))
    z_stat = diff_mean / se_diff
    cl_ll = diff_mean - stats.norm.ppf(1 - alpha/2) * se_diff
    cl_ul = diff_mean + stats.norm.ppf(1 - alpha/2) * se_diff
    # pval = 2 * (1 - stats.norm.cdf(abs(z_stat)))

    return {
        'group1': group1,
        'group2': group2,
        '평균차이': diff_mean,
        '신뢰구간_하한': cl_ll,
        '신뢰구간_상한': cl_ul,
        'z_통계량': z_stat,
        'p값': pval
    }

In [33]:
stats_df2 = pd.DataFrame([analysis_pair_wise_abt_by_parametric(sample_df, *comb) for comb in pairwise_combinations])
stats_df2

,group1,group2,평균차이,신뢰구간_하한,신뢰구간_상한,z_통계량,p값
0,A,B,0.002059,-0.003201,0.007320,0.767177,0.413852
1,A,C,0.000977,-0.004285,0.006239,0.363804,0.698301
2,B,C,-0.001082,-0.006316,0.004151,-0.405320,0.666048


다중 검정

In [34]:
# ANOVA 분석 진행
from scipy.stats import f_oneway

stat, p = f_oneway(
    sample_df[sample_df['grp'] == 'A']['amount'],
    sample_df[sample_df['grp'] == 'B']['amount'],
    sample_df[sample_df['grp'] == 'C']['amount']
)

# 가설 검정
alpha = 0.05
if p_value > alpha:
    print("귀무가설 채택: 세 그룹 간의 차이가 통계적으로 유의미하지 않음")
else:
    print("귀무가설 기각: 세 그룹 간의 차이가 통계적으로 유의미함")

귀무가설 기각: 세 그룹 간의 차이가 통계적으로 유의미함


사후 검정 - Tukey 검정

* 본 테스트에서는 유의미한 결과가 안나왔지만, ANOVA 가 유의하다면, Tukey 검정을 통해,
  
* 어떤 그룹과 그룹의 차이가 유의한지 확인하는 사후 검정을 진행한다.

In [299]:
from statsmodels.sandbox.stats.multicomp import MultiComparison
sample_df['amount'] = sample_df['amount'].astype(float)  # ensure 'amount' is float
res = MultiComparison(sample_df['amount'], sample_df['grp']).tukeyhsd(alpha=0.05)
res_df = res.summary()
res_df

group1,group2,meandiff,p-adj,lower,upper,reject
A,B,11247.1608,0.0,11200.9697,11293.3519,True
A,C,11255.3678,0.0,11209.2669,11301.4687,True
B,C,8.207,0.9083,-37.8468,54.2608,False


비모수 검정
  
   * N < 5000 이고 shapiro-test p-value < 0.05 이면, 비모수 검정이 적합함

    * 2가지 테스트가 존재
    * Mann-Whitney U test : ABT 에서 진행
    * Kruskal-Wallis H Test : MVT 에서 진행

In [ ]:
from lib.mann_whitney import MannWhitney

In [301]:
def analysis_pair_wise_abt_by_nonparametric(df, group1, group2, adj_alpha=0.05):
    filtered_df = df[(df['grp'] == group1) | (df['grp'] == group2)]
    data1 = filtered_df[filtered_df['grp'] == group1]['amount']
    data2 = filtered_df[filtered_df['grp'] == group2]['amount']
    mu1 = np.mean(data1)
    mu2 = np.mean(data2)
    stat, p_value = MannWhitney(data1, data2, sig=adj_alpha).test()
    
    result = {
        'group1': group1,
        'group2': group2,
        '평균차이': mu1 - mu2,
        '신뢰구간_하한': None,  # 비모수 검정의 신뢰구간 상하한 계산량은 O(n^2)로 너무 크기 때문에 생략
        '신뢰구간_상한': None,
        'u_통계량': stat,
        'p값': p_value,
    }
    return result

In [302]:
adj_alpha = 0.05 / len(pairwise_combinations) # 본페로니 교정
stats_df3 = pd.DataFrame([analysis_pair_wise_abt_by_nonparametric(sample_df, *comb, adj_alpha=adj_alpha) for comb in pairwise_combinations])
stats_df3

,group1,group2,평균차이,신뢰구간_하한,신뢰구간_상한,u_통계량,p값
0,A,B,-11247.160790,None,None,539495552.0,0.109160
1,A,C,-11255.367775,None,None,539495552.0,0.000003
2,B,C,-8.206985,None,None,543906162.0,0.081319


MVT - 비모수 사후분석

In [71]:
data = sample_df.groupby('grp')['amount'].apply(lambda x : list(x)).tolist()

In [72]:
# Kruskal-Wallis H Test 진행

stat, p_value = stats.kruskal(*data)

print(f"Kruskal-Wallis Test Statistic: {stat}")
print(f"P-value: {p_value}")

# 가설 검정
alpha = 0.05
if p_value > alpha:
    print("귀무가설 채택: 세 그룹 간의 차이가 통계적으로 유의미하지 않음")
else:
    print("귀무가설 기각: 세 그룹 간의 차이가 통계적으로 유의미함")

Kruskal-Wallis Test Statistic: 54063.609951711114
P-value: 0.0
귀무가설 기각: 세 그룹 간의 차이가 통계적으로 유의미함
